# Notebook 05: Electron Transfer (Marcus Theory)

## 1. Introduction: The Simplest Reaction

Electron transfer (ET) is arguably the simplest chemical reaction: an electron jumps from a donor (D) to an acceptor (A). No bonds break, no atoms move significantly. Yet, it powers photosynthesis, respiration, and batteries.

**Rudolph Marcus** won the Nobel Prize for explaining how the rate of ET depends on the driving force and the solvent environment.

### Learning Objectives
1.  Understand the concept of Reorganization Energy ($\lambda$).
2.  Derive the Marcus equation from intersecting parabolas.
3.  Explain the "Inverted Region" where reactions slow down as they become more exothermic.
4.  Describe the distance dependence of electron tunneling.

In [ ]:
# ============================================================
# GOOGLE COLAB SETUP
# ============================================================
import sys
import os

# Check if running in Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("=" * 60)
    print("RUNNING IN GOOGLE COLAB")
    print("=" * 60)

    # Clone repository to access images
    repo_url = "https://github.com/mcbadlon31/Reaction-Dynamics-Physical-Chemistry.git"

    print(f"\nCloning repository: {repo_url}")
    print("This may take a minute...")

    !git clone {repo_url} --depth 1 --quiet

    # Change to repository directory
    os.chdir('Reaction-Dynamics-Physical-Chemistry')

    # Install additional packages if needed
    print("\nInstalling additional packages...")
    !pip install -q seaborn plotly ipywidgets

    print("\n" + "=" * 60)
    print("[SUCCESS] Colab setup complete!")
    print("=" * 60)
    print(f"Current directory: {os.getcwd()}")
    print("\nYou can now run all cells normally.")
    print("Images will load from the cloned repository.")

else:
    print("=" * 60)
    print("RUNNING IN LOCAL JUPYTER ENVIRONMENT")
    print("=" * 60)
    print("\nNo setup needed - using local files")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants
import ipywidgets as widgets
from IPython.display import display

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'figure.dpi': 120,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'lines.linewidth': 2,
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'DejaVu Sans'],
    'grid.alpha': 0.3
})

k_B = constants.Boltzmann
h = constants.Planck
T = 298

print("Libraries loaded.")

## 2. The Franck-Condon Principle and Reorganization

Electrons move much faster than nuclei. When an electron jumps, the nuclei (solvent and ligands) don't have time to move. 
-   **Reactant State (R)**: D + A, solvent oriented around neutral/charged species.
-   **Product State (P)**: D$^+$ + A$^-$, solvent needs to reorient to stabilize new charges.

Because of the Franck-Condon principle, ET can only occur when the nuclear configuration of the Reactant state happens to have the same energy as the Product state. The energy cost to distort the Reactant state to this configuration is the **activation energy**.

### Reorganization Energy ($\lambda$)
$\lambda$ is the energy required to distort the reactant equilibrium geometry to the product equilibrium geometry *without* transferring the electron.
On the Marcus diagram, it corresponds to the vertical energy difference between the product curve at the reactant coordinate ($q=0$) and the product curve minimum.

$$ \lambda = \lambda_{inner} + \lambda_{outer} $$
-   **Inner**: Bond changes (vibrations).
-   **Outer**: Solvent reorientation.

In [ ]:
class ReorganizationEnergyExplorer:
    """Explore contributions to reorganization energy"""
    
    def __init__(self):
        self.setup_widgets()
        self.setup_display()
        
    def setup_widgets(self):
        self.solvent_dropdown = widgets.Dropdown(
            options={'Water (Polar)': 80.0, 'Acetonitrile (Polar)': 37.5, 'Benzene (Non-polar)': 2.3},
            value=80.0,
            description='Solvent:'
        )
        self.radius_slider = widgets.FloatSlider(
            min=2.0, max=10.0, step=0.5, value=4.0, 
            description='Ion Radius (Å)'
        )
        
    def plot(self, epsilon_s, radius):
        # Simplified Born model for Outer Sphere Reorganization
        # lambda_outer ~ (1/n^2 - 1/epsilon_s) * (1/r)
        # n (refractive index) approx 1.4 for most organics
        n = 1.4
        epsilon_op = n**2
        
        # Factor C depends on 1/r (simplified)
        # lambda_o (eV) approx 14.4 * (1/ep_op - 1/ep_s) * (1/2r1 + 1/2r2 - 1/R)
        # Let's assume r1=r2=radius, R=2*radius (contact)
        term = (1/epsilon_op - 1/epsilon_s)
        geom = (1/radius) # simplified
        lambda_outer = 14.4 * term * geom * 0.5 # eV approx
        
        # Inner sphere (constant for this demo)
        lambda_inner = 0.3 # eV
        
        total_lambda = lambda_inner + lambda_outer
        
        # Plot
        plt.figure(figsize=(8, 5))
        plt.bar(['Inner (Bond)', 'Outer (Solvent)'], [lambda_inner, lambda_outer], 
                color=['brown', 'skyblue'])
        
        plt.ylabel('Energy (eV)')
        plt.title(f'Reorganization Energy Components (Total $\lambda$ = {total_lambda:.2f} eV)')
        plt.ylim(0, 2.0)
        plt.grid(axis='y', alpha=0.3)
        
        # Add text
        plt.text(1, lambda_outer + 0.1, f"{lambda_outer:.2f} eV", ha='center')
        plt.text(0, lambda_inner + 0.1, f"{lambda_inner:.2f} eV", ha='center')
        
        plt.show()
        
        print(f"Solvent Dielectric Constant: {epsilon_s}")
        print(f"Pekar Factor (1/n² - 1/ε): {term:.3f}")
        
    def setup_display(self):
        widgets.interact(self.plot, epsilon_s=self.solvent_dropdown, radius=self.radius_slider)

print("\n💧 Reorganization Energy Explorer:")
reorg_explorer = ReorganizationEnergyExplorer()

### ❓ Concept Check

<details>
<summary><strong>Q: Why is the reorganization energy $\lambda$ usually larger in polar solvents (like water) than in non-polar solvents (like hexane)?</strong></summary>

Polar solvent molecules have permanent dipoles that must reorient significantly to accommodate the change in charge distribution during electron transfer. This requires more energy than the slight polarization of non-polar solvent molecules.
</details>

## 3. The Marcus Equation

We model the free energy of Reactants ($G_R$) and Products ($G_P$) as harmonic oscillators (parabolas) along a reaction coordinate $q$.

$$ G_R(q) = \lambda q^2 $$
$$ G_P(q) = \lambda (q-1)^2 + \Delta G^\circ $$

The transition state is the intersection point ($q^\ddagger$). Equating $G_R(q^\ddagger) = G_P(q^\ddagger)$ and solving for the activation energy $\Delta^\ddagger G = G_R(q^\ddagger)$ yields:

$$ \Delta^\ddagger G = \frac{(\Delta G^\circ + \lambda)^2}{4\lambda} $$

The rate constant is then:
$$ k_{ET} = \kappa \nu_N e^{-\frac{(\Delta G^\circ + \lambda)^2}{4\lambda RT}} $$

In [ ]:
def calculate_marcus(dG0_eV, lambda_eV):
    # Convert to J
    dG0 = dG0_eV * constants.elementary_charge
    lam = lambda_eV * constants.elementary_charge
    
    # Activation Energy
    dG_act = (dG0 + lam)**2 / (4 * lam)
    
    print(f"Driving Force dG0: {dG0_eV} eV")
    print(f"Reorganization Energy lambda: {lambda_eV} eV")
    print(f"Activation Energy dG_act: {dG_act / constants.elementary_charge:.3f} eV")
    
    if abs(dG0 + lam) < 0.01:
        print("Regime: Activationless (Maximum Rate)")
    elif -dG0 < lam:
        print("Regime: Normal")
    else:
        print("Regime: Inverted")

widgets.interact(calculate_marcus, 
                 dG0_eV=widgets.FloatSlider(min=-3.0, max=1.0, step=0.1, value=-0.5, description='dG0 (eV)'),
                 lambda_eV=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='lambda (eV)'));

### ❓ Concept Check

<details>
<summary><strong>Q: Under what condition does the activation energy become zero?</strong></summary>

When $-\Delta G^\circ = \lambda$. In this case, the driving force exactly matches the reorganization energy, and the intersection of the parabolas occurs at the very bottom of the reactant well. This is the "activationless" regime.
</details>

## 4. Visualizing Marcus Parabolas

The plot below shows the Reactant (Blue) and Product (Red) free energy curves.
-   **Normal Region**: $-\Delta G^\circ < \lambda$. Activation energy decreases as driving force increases.
-   **Activationless**: $-\Delta G^\circ = \lambda$. Intersection is at the bottom of the R parabola. Rate is max.
-   **Inverted Region**: $-\Delta G^\circ > \lambda$. Intersection moves *up* the parabola. Rate decreases as driving force increases!

![Marcus Parabolas](images/marcus_parabolas_annotated.png)

In [ ]:
def plot_marcus_parabolas(dG0_eV, lambda_eV):
    q = np.linspace(-0.5, 1.5, 100)
    
    # Reactant parabola (centered at 0)
    G_R = lambda_eV * q**2
    
    # Product parabola (centered at 1, shifted by dG0)
    G_P = lambda_eV * (q - 1)**2 + dG0_eV
    
    # Intersection (Transition State)
    q_TS = (lambda_eV + dG0_eV) / (2 * lambda_eV)
    E_TS = lambda_eV * q_TS**2
    
    plt.figure(figsize=(8, 6))
    plt.plot(q, G_R, 'b-', label='Reactant (D + A)')
    plt.plot(q, G_P, 'r-', label='Product (D+ + A-)')
    
    # Plot TS
    plt.plot(q_TS, E_TS, 'ko', markersize=10, label='Transition State')
    
    # Annotations
    plt.annotate(f'dG_act = {E_TS:.2f} eV', xy=(q_TS, E_TS), xytext=(q_TS, E_TS + 0.5),
                 arrowprops=dict(facecolor='black', shrink=0.05))
    
    # Reorganization Energy lambda
    # Vertical line at q=0 from G_P_min (dG0) to G_P(0) (lambda + dG0)
    plt.plot([0, 0], [dG0_eV, lambda_eV + dG0_eV], 'k--', alpha=0.5)
    plt.annotate(r'$\lambda$', xy=(0, lambda_eV/2 + dG0_eV), xytext=(-0.1, lambda_eV/2 + dG0_eV), fontsize=12)
    
    plt.xlabel('Reaction Coordinate q')
    plt.ylabel('Free Energy (eV)')
    plt.title('Marcus Parabolas')
    plt.legend()
    plt.grid(True)
    plt.ylim(min(dG0_eV, 0) - 0.5, max(lambda_eV, abs(dG0_eV)) + 1.0)
    plt.show()

widgets.interact(plot_marcus_parabolas, 
                 dG0_eV=widgets.FloatSlider(min=-3.0, max=1.0, step=0.1, value=-0.5, description='dG0 (eV)'),
                 lambda_eV=widgets.FloatSlider(min=0.5, max=2.0, step=0.1, value=1.0, description='lambda (eV)'));

## 5. The Inverted Region

This counter-intuitive prediction was one of the triumphs of Marcus theory. It explains why highly exothermic charge recombination reactions (e.g., in photosynthesis) are slow, preventing energy waste.

In [ ]:
def plot_inverted_region(lambda_eV):
    dG0 = np.linspace(-3.0, 0.5, 100)  # eV
    
    # Activation energy
    dG_act = (dG0 + lambda_eV)**2 / (4 * lambda_eV)
    
    # Rate constant (proportional to exp(-dG_act/kT))
    kT = k_B * T / constants.elementary_charge  # eV
    ln_k = -dG_act / kT
    
    plt.figure(figsize=(8, 5))
    plt.plot(-dG0, ln_k, 'g-', linewidth=2)
    
    # Mark lambda
    plt.axvline(lambda_eV, color='k', linestyle='--', label=f'lambda = {lambda_eV} eV')
    
    plt.xlabel('Driving Force -dG0 (eV)')
    plt.ylabel('ln(Rate Constant)')
    plt.title('Marcus Inverted Region')
    plt.legend()
    plt.grid(True)
    plt.show()

widgets.interact(plot_inverted_region, 
                 lambda_eV=widgets.FloatSlider(min=0.5, max=2.0, step=0.1, value=1.0, description='lambda (eV)'));

### ❓ Concept Check

<details>
<summary><strong>Q: Why is the Inverted Region important for biology?</strong></summary>

In photosynthesis, after light creates a charge-separated state, the system wants to prevent the charges from recombining (which would waste the energy as heat). Recombination is highly exothermic, pushing it into the **inverted region**, where the rate is slow. This allows the useful forward electron transfer steps to compete effectively.
</details>

## 6. Distance Dependence and Biological ET

The electron must tunnel through the medium. The rate decreases exponentially with distance:
$$ k_{ET} \propto e^{-\beta (r - r_0)} $$
where $\beta$ describes the nature of the medium (approx 1.4 $\mathring{A}^{-1}$ for proteins).

In biological systems (like the Electron Transport Chain), electrons hop between cofactors (hemes, FeS clusters) arranged in a chain to travel long distances efficiently.

In [ ]:
class TunnelingVisualizer:
    """Visualize quantum tunneling through a barrier"""
    
    def __init__(self):
        self.setup_widgets()
        self.setup_display()
        
    def setup_widgets(self):
        self.width_slider = widgets.FloatSlider(
            min=1.0, max=10.0, step=0.5, value=5.0,
            description='Width (Å)'
        )
        self.height_slider = widgets.FloatSlider(
            min=1.0, max=10.0, step=0.5, value=5.0,
            description='Barrier (eV)'
        )
        self.energy_slider = widgets.FloatSlider(
            min=0.5, max=9.5, step=0.5, value=2.0,
            description='Energy (eV)'
        )
        
    def plot(self, width, V0, E):
        # Constants
        hbar = constants.hbar
        m_e = constants.m_e
        eV = constants.elementary_charge
        Angstrom = 1e-10
        
        # Convert to SI
        a = width * Angstrom
        V0_J = V0 * eV
        E_J = E * eV
        
        # x range (Angstroms)
        x = np.linspace(-10, 20, 1000)
        x_SI = x * Angstrom
        
        # Potential
        V = np.zeros_like(x)
        mask_barrier = (x >= 0) & (x <= width)
        V[mask_barrier] = V0
        
        # Wavefunction (approximate schematic)
        psi = np.zeros_like(x)
        
        # Region 1: x < 0 (Incident + Reflected)
        k1 = np.sqrt(2 * m_e * E_J) / hbar
        # Region 2: 0 <= x <= a (Tunneling)
        if V0_J > E_J:
            kappa = np.sqrt(2 * m_e * (V0_J - E_J)) / hbar
            # Transmission probability (approx)
            T_prob = np.exp(-2 * kappa * a)
        else:
            kappa = 0 # Oscillatory
            T_prob = 1.0 # Simplified
            
        # Region 3: x > a (Transmitted)
        amp_trans = np.sqrt(T_prob)
        
        # Construct schematic wavefunction for visualization
        # Real part only
        mask_1 = x < 0
        mask_2 = (x >= 0) & (x <= width)
        mask_3 = x > width
        
        psi[mask_1] = np.cos(k1 * x_SI[mask_1] * 1e10) # Fast oscillation
        
        if V0_J > E_J:
            # Decay
            # Match boundary at x=0 (approx)
            psi[mask_2] = np.exp(-kappa * x_SI[mask_2]) 
            # Scale to match cos(0)=1
            
            # Transmitted
            # Match boundary at x=a
            val_at_a = np.exp(-kappa * a)
            phase_shift = k1 * a
            psi[mask_3] = val_at_a * np.cos(k1 * (x_SI[mask_3] - a))
        else:
            # Above barrier
            k2 = np.sqrt(2 * m_e * (E_J - V0_J)) / hbar
            psi[mask_2] = np.cos(k2 * x_SI[mask_2])
            psi[mask_3] = np.cos(k1 * (x_SI[mask_3] - a) + k2*a)

        # Plotting
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Plot Potential
        ax.plot(x, V, 'k-', linewidth=2, label='Potential Barrier')
        ax.fill_between(x, 0, V, color='gray', alpha=0.2)
        
        # Plot Energy Level
        ax.axhline(E, color='g', linestyle='--', label='Electron Energy')
        
        # Plot Wavefunction (shifted by E for visibility)
        scale = 1.0
        ax.plot(x, E + scale * psi, 'r-', linewidth=1.5, label='Wavefunction $\psi$')
        
        ax.set_xlabel('Position (Å)')
        ax.set_ylabel('Energy (eV)')
        ax.set_title(f'Quantum Tunneling (Transmission T ≈ {T_prob:.2e})')
        ax.legend(loc='upper right')
        ax.set_ylim(0, max(V0, E) * 1.5)
        ax.grid(True, alpha=0.3)
        
        plt.show()
        
        print(f"Barrier Width: {width} Å")
        print(f"Barrier Height: {V0} eV")
        print(f"Electron Energy: {E} eV")
        if V0 > E:
            print(f"Tunneling Probability: {T_prob:.2e}")
            print("The electron 'tunnels' through the forbidden region!")
        else:
            print("Energy > Barrier: Classical transmission allowed.")
            
    def setup_display(self):
        widgets.interact(self.plot, width=self.width_slider, V0=self.height_slider, E=self.energy_slider)

print("\n👻 Quantum Tunneling Visualizer:")
tunnel_viz = TunnelingVisualizer()

In [ ]:
class BiologicalETChain:
    """Simulate electron hopping in a protein chain"""
    
    def __init__(self):
        self.setup_widgets()
        self.setup_display()
        
    def setup_widgets(self):
        self.distance_slider = widgets.FloatSlider(
            min=5.0, max=20.0, step=1.0, value=10.0, 
            description='Spacing (Å)'
        )
        
    def plot(self, spacing):
        # Total distance to cover: 60 Angstroms
        total_dist = 60
        num_steps = int(total_dist / spacing)
        
        # Rate for one step
        beta = 1.4 # A^-1
        r0 = 3.0 # contact
        k_step_rel = np.exp(-beta * (spacing - r0))
        
        # Rate for single long jump
        k_jump_rel = np.exp(-beta * (total_dist - r0))
        
        # Total time (inverse of rate)
        # Time_chain = num_steps * (1/k_step)
        # Time_jump = 1/k_jump
        
        # Relative advantage
        advantage = k_step_rel / k_jump_rel
        
        # Visualization
        plt.figure(figsize=(10, 4))
        
        # Draw cofactors
        x_pos = np.linspace(0, total_dist, num_steps + 1)
        y_pos = np.zeros_like(x_pos)
        
        plt.plot(x_pos, y_pos, 'o-', markersize=15, color='green', label='Cofactors')
        
        # Annotate
        plt.title(f'Electron Transport Chain (Total Distance: {total_dist} Å)')
        plt.xlabel('Distance (Å)')
        plt.yticks([])
        plt.ylim(-1, 1)
        
        # Arrows
        for i in range(len(x_pos)-1):
            plt.annotate('', xy=(x_pos[i+1], 0.2), xytext=(x_pos[i], 0.2),
                         arrowprops=dict(arrowstyle='->', color='orange', lw=2))
            
        plt.show()
        
        print(f"Step Size: {spacing} Å")
        print(f"Number of Steps: {num_steps}")
        print(f"Relative Rate (Hopping): {k_step_rel:.2e}")
        print(f"Relative Rate (Single Jump): {k_jump_rel:.2e}")
        print(f"\n⚡ Hopping Advantage Factor: 10^{np.log10(advantage):.1f}")
        print("Nature uses chains of cofactors to prevent exponential decay of the rate!")
        
    def setup_display(self):
        widgets.interact(self.plot, spacing=self.distance_slider)

print("\n🧬 Biological Electron Transport Chain:")
bio_chain = BiologicalETChain()

## Summary

1.  **Marcus Theory**: Connects kinetics (rate) to thermodynamics (driving force) and structure (reorganization).
2.  **Parabolas**: The intersection of reactant and product free energy curves determines the barrier.
3.  **Inverted Region**: Increasing driving force beyond $\lambda$ slows the reaction.
4.  **Tunneling**: ET is a quantum mechanical tunneling phenomenon, sensitive to distance.
5.  **Biological Design**: Nature uses chains of cofactors to transport electrons over long distances, avoiding the steep penalty of single long-distance tunneling.